In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from datasets import load_dataset

# Step 1: Define a custom Dataset class
class MaterialDataset(Dataset):
    def __init__(self, split, transform=None):
        self.dataset = load_dataset("Erland/AI701_project", split=split)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['resized_image']
        label = item['label']  # Adjust key if necessary
        if self.transform:
            image = self.transform(image)
        return image, label

# Step 2: Define image transformations
train_transforms = transforms.Compose([
    transforms.ToTensor(),
])

# Step 3: Load datasets and create DataLoaders
train_dataset = MaterialDataset(split='train', transform=train_transforms)
test_dataset = MaterialDataset(split='test', transform=train_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
model = models.resnet101(pretrained=True)
model

/home/rifo.genadi/.conda/envs/vlm/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rifo.genadi/.conda/envs/vlm/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [2]:
# Step 4: Load a pre-trained ResNet model
model = models.efficientnet_b0(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Step 5: Modify the final layer to match the number of classes
num_classes = len(set(train_dataset.dataset['label']))
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
# model.fc = nn.Linear(model.fc.in_features, num_classes)

# Step 6: Set up the device, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 7: Training loop
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    train_loss = running_loss / len(train_loader)

    # # Validation phase
    # model.eval()
    # correct_val = 0
    # total_val = 0
    # val_loss = 0.0

    # with torch.no_grad():
    #     for images, labels in val_loader:
    #         images = images.to(device)
    #         labels = labels.to(device)

    #         outputs = model(images)
    #         loss = criterion(outputs, labels)

    #         val_loss += loss.item()

    #         _, predicted = torch.max(outputs.data, 1)
    #         total_val += labels.size(0)
    #         correct_val += (predicted == labels).sum().item()

    # val_accuracy = 100 * correct_val / total_val
    # val_loss = val_loss / len(val_loader)

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
        #   f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%'
          )

# Step 8: Testing phase
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f'Test Accuracy: {test_accuracy:.2f}%')

/home/rifo.genadi/.conda/envs/vlm/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rifo.genadi/.conda/envs/vlm/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20, Train Loss: 1.5947, Train Acc: 46.02%, 
Epoch 2/20, Train Loss: 1.3412, Train Acc: 54.06%, 
Epoch 3/20, Train Loss: 1.2481, Train Acc: 56.94%, 
Epoch 4/20, Train Loss: 1.2027, Train Acc: 57.57%, 
Epoch 5/20, Train Loss: 1.1589, Train Acc: 60.31%, 
Epoch 6/20, Train Loss: 1.1362, Train Acc: 60.43%, 
Epoch 7/20, Train Loss: 1.1038, Train Acc: 60.95%, 
Epoch 8/20, Train Loss: 1.0871, Train Acc: 61.98%, 
Epoch 9/20, Train Loss: 1.0647, Train Acc: 62.76%, 
Epoch 10/20, Train Loss: 1.0537, Train Acc: 62.98%, 
Epoch 11/20, Train Loss: 1.0593, Train Acc: 62.96%, 
Epoch 12/20, Train Loss: 1.0374, Train Acc: 63.34%, 
Epoch 13/20, Train Loss: 1.0335, Train Acc: 63.69%, 
Epoch 14/20, Train Loss: 1.0235, Train Acc: 64.02%, 
Epoch 15/20, Train Loss: 1.0208, Train Acc: 63.97%, 
Epoch 16/20, Train Loss: 1.0193, Train Acc: 65.00%, 
Epoch 17/20, Train Loss: 1.0154, Train Acc: 63.40%, 
Epoch 18/20, Train Loss: 1.0141, Train Acc: 64.38%, 
Epoch 19/20, Train Loss: 1.0026, Train Acc: 64.43%, 
Ep